In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


local
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved.cu -o /mnt/d//packing/temp/pack_cuda_crystal.cubin
ptxas info    : 0 bytes gmem, 1172 bytes cmem[3]
ptxas info    : Compiling entry function 'multi_boundary_distance_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_distance_list_total
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 36 registers, used 1 barriers, 404 bytes cmem[0], 16 bytes cmem[2]
ptxas info    : Compile time = 9.104 ms
ptxas info    : Compiling entry function 'multi_boundary_list_total' for 'sm_89'
ptxas info  

In [ ]:
%load_ext pyinstrument

In [ ]:
#%%pyinstrument
#kgs.profiling=True
# %matplotlib widget
#kgs.enable_module_profiling(kgs)
import pack_ga3
import pack_runner
fastMode = False
kgs.debugging_mode = 1

for seed in range(1):

    runner = pack_ga3.baseline_symmetry_180()
    pack_runner.set_connectivity_pattern(runner, '', 6) #tree
    print(type(runner.ga))
    runner.ga.connect_siblings = True
    runner.ga.parent_child_depth = 3
    runner.ga.parent_child_one_way = True    
    runner.ga.allow_reset_ratio = 1.
    runner.ga.diversity_reset_threshold = -1.
    runner.ga.ga_base.reset_check_generations = 100
    runner.ga.ga_base.reset_check_generations_ratio = 0.
    runner.ga.ga_base.always_allow_mate_with_better = True
    runner.rough_relaxers[0].max_step = 1e-1
    print('still scale reset checks')

    runner.ga.ga_base.N_trees_to_do = 68
    runner.diagnostic_plot = True
    runner.seed = seed
    runner.plot_every = 10
    #runner.filter_before_rough = 1.

    runner.run()

<class 'pack_ga3.GAMultiTree'>
still scale reset checks
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

GAMultiTree structure (31 islands, depth 5):
└── 0
    ├── 1
    │   ├── 3
    │   │   ├── 7
    │   │   │   ├── 15
    │   │   │   └── 16
    │   │   └── 8
    │   │       ├── 17
    │   │       └── 18
    │   └── 4
    │       ├── 9
    │       │   ├── 19
    │       │   └── 20
    │       └── 10
    │           ├── 21
    │           └── 22
    └── 2
        ├── 5
        │   ├── 11
        │   │   ├── 23
        │   │   └── 24
        │   └── 12
        │       ├── 25
        │       └── 26
        └── 6
            ├── 13
            │   ├── 27
            │   └── 28
            └── 14
                ├── 29
                └── 30

Connectivity (each island connects to parent, children, and sibling):
  Island 0: → []
  Island 1: → [0, 2]
  Island 2: → [0, 1]
  Island 3: →

/mnt/d/packing/code/analysis/../core/pack_dynamics.py:61: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  x0 = from_dlpack(x0.toDlpack())
/mnt/d/packing/code/analysis/../core/pack_dynamics.py:93: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  return from_dlpack(tmp_cost[:N].toDlpack()), from_dlpack(res.toDlpack())
